In [12]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [13]:
def do_etsy_request(research:str, page:int):
    url = 'https://www.etsy.com/fr/search?q='
    research = research.replace(' ', '%20')
    url = url + research + '&page='+ str(page)
    return url

In [14]:
def page_loader(url:str):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.etsy.com/',  # Referer spécifique à Etsy
    'Cache-Control': 'max-age=0',
    'DNT': '1',  # Do Not Track
    'Upgrade-Insecure-Requests': '1'
    }
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text,'html')
    return soup, page.status_code

In [31]:
def scrapp(reseach:str, maxpage:int):

    column = ["Lien image","Nom du produit","Note","Nombre de votes","Nom de la boutique","Prix","Symbol","Lien vers le produit"]
    column_page= ["Num_page", "URL", "Status_HTTP", "Nb Retry", "Nb_Products"]
    rows=[]
    pages=[]
    nb_page_found = 0
    nb_page_ignore = 0
    nb_product_find = 0
    nb_product_ignore =0

    for page in range(1, maxpage + 1):
        page_data=[]
        url = do_etsy_request(reseach, page)

        for retry in range(1, 15):
            soup, status = page_loader(url=url)
            if status == 200 :
                # print(str(page) + " | "+ url + " -- " + str(status) + "---> [" + str(retry) + "]")
                page_data.append(page)
                page_data.append(url)
                page_data.append(status)
                page_data.append(retry)
                nb_page_found += 1
                break

        if status == 200:
            search_results = soup.find('div', attrs={'data-search-results':True})
            elements = search_results.find_all('li')
            product_find = 0

            for el in elements:
                if not el.find('img', class_='wt-width-full') == None:
                    if 'style' not in el.attrs:
                        nb_product_find += 1
                        row = []

                        # Lien vers l'image
                        img_link = el.find('img', class_='wt-width-full').get('src', '')
                    
                        # Nom du produit
                        product_name = el.find('h3', class_='v2-listing-card__title').text.strip()
                    
                        # Note et votes
                        if not el.find('div', class_='wt-align-items-center wt-max-height-full wt-display-flex-xs flex-direction-row-xs wt-text-title-small wt-no-wrap') == None:
                            note = el.find('div', class_='wt-align-items-center wt-max-height-full wt-display-flex-xs flex-direction-row-xs wt-text-title-small wt-no-wrap').text.strip()
                            note,_ , votes = note.split('\n')
                        else : 
                            note = "NA"
                            votes = "NA"
                    
                        # Nom de la boutique
                        all_span = el.find_all('p')[1].find_all("span")
                        if len(all_span) > 3:
                            shop_name = all_span[3].text.strip()
                        else:
                            shop_name="Not Found"
                    
                        # Prix
                        price_value = el.find('span', class_='currency-value').text.strip()
                        price_symbol = el.find('span', class_='currency-symbol').text.strip()

                        # Lien vers le produit
                        product_link = el.find('a', class_='listing-link').get('href', '')

                        # Afficher les informations
                        # print(f"Lien vers l'image : {img_link}")
                        row.append(img_link)
                        # print(f"Nom du produit : {product_name}")
                        row.append(product_name)
                        # print(f"Note : {note}")
                        row.append(note)
                        # print(f"Nombre de votes : {votes}")
                        row.append(votes)
                        # print(f"Nom de la boutique : {shop_name}")
                        row.append(shop_name)
                        # print(f"Prix : {price_value} {price_symbol}")
                        row.append(price_value)
                        row.append(price_symbol)
                        # print(f"Lien vers le produit : {product_link}")
                        row.append(product_link)
                        # print('-' * 50)  # Pour séparer les informations de différents produits
                        product_find += 1
                        rows.append(row)
                    else:
                        nb_product_ignore += 1
                        # pass
                else:
                    pass
            else:
                nb_page_ignore += 1
                # pass

        percent = round((page/maxpage)*100,0)
        print(f'%{percent} | {page} pages done on {(maxpage)} pages ')
        page_data.append(product_find)
        pages.append(page)

    output = pd.DataFrame(rows, columns=column)
    output_page = pd.DataFrame(page_data, column_page)
    output_kpi = [nb_page_found, nb_page_ignore, nb_product_find, nb_product_ignore]
    # print(f"RESULTAT SCRAPPER | Nb page found [{nb_page_found}]| Nb page ignored [{nb_page_ignore}]| Nb products found [{nb_product_find}]| Nb product ignored [{nb_product_ignore}]")
    return output, output_page, output_kpi

In [32]:
MA_RECHERCHE = "couverture chaude bebe"
MAX_PAGE = 5

file_name = "etsy_data_" + MA_RECHERCHE.replace(' ','_') + '.csv'

ma_data, kpi = scrapp(MA_RECHERCHE, MAX_PAGE)
# ma_data.to_csv(file_name, sep=";",encoding="utf-8",index=False)

1 | https://www.etsy.com/fr/search?q=couverture%20chaude%20bebe&page=1 -- 200---> [1]
%20.0 | 1 pages done on 5 pages 
2 | https://www.etsy.com/fr/search?q=couverture%20chaude%20bebe&page=2 -- 200---> [3]
%40.0 | 2 pages done on 5 pages 
3 | https://www.etsy.com/fr/search?q=couverture%20chaude%20bebe&page=3 -- 200---> [1]
%60.0 | 3 pages done on 5 pages 
4 | https://www.etsy.com/fr/search?q=couverture%20chaude%20bebe&page=4 -- 200---> [1]
%80.0 | 4 pages done on 5 pages 
5 | https://www.etsy.com/fr/search?q=couverture%20chaude%20bebe&page=5 -- 200---> [2]
%100.0 | 5 pages done on 5 pages 


ValueError: too many values to unpack (expected 2)